# PREPARATION

## Import Required Library

In [1]:
import math
from itertools import permutations
from itertools import product
import pandas as pd
import numpy as np
import ipywidgets as widgets
from IPython.display import display
import json

## Global Helper Function

In [2]:
def create_criteria_decision_form() -> None: 
    global total_criteria
    
    dropdowns = []
    for i in range(total_criteria):
        label = widgets.Label(value=f"C{i+1}", layout=widgets.Layout(width="50px"))
    
        dropdown = widgets.Dropdown(options=['Benefit', 'Cost'], layout=widgets.Layout(width="100px"))
        dropdowns.append(dropdown)
    
        display(widgets.HBox([label, dropdown]))

    def get_dropdown_values():
        return [dropdown.value for dropdown in dropdowns]
    
    output = widgets.Output()
    
    def set_criteria(_):
        global criterias
        
        criterias['type'] = get_dropdown_values()
            
        with output:
            output.clear_output(wait=True)
            print('Criterias')
            print(criterias)
        
    set_criteria_btn = widgets.Button(description="Set Criteria!")
    set_criteria_btn.on_click(set_criteria)

    display(set_criteria_btn, output)

In [3]:
def create_matrix_form() -> None:
    global total_criteria, total_alternative, criteria_labels, alternative_labels

    inputs = []
    for i in range(total_alternative):
        rows = []
        for c in range(total_criteria):
            rows.append(widgets.FloatText(value=0, layout=widgets.Layout(width="100px")))
        inputs.append(rows)

    header_row = []
    for i in range(len(criteria_labels) + 1):
        header_row.append(widgets.Label(
            value='' if i == 0 else criteria_labels[i - 1],
            layout=widgets.Layout(width="100px")
        ))
    
    input_rows = []
    for r in range(total_alternative):
        row_widgets = [widgets.Label(value=alternative_labels[r], layout=widgets.Layout(width="50px"))] + inputs[r]
        input_rows.append(widgets.HBox(row_widgets))

    def get_matrix():
        return np.array([[cell.value for cell in row] for row in inputs])

    output = widgets.Output()
    
    def update_matrix(_):
        global decision_matrix
        
        decision_matrix.loc[:, :] = get_matrix()

        with output:
            print("Decision matrix:")
            print(decision_matrix)

    set_matrix_btn = widgets.Button(description="Set Matrix!")
    set_matrix_btn.on_click(update_matrix)

    display(widgets.HBox(header_row))
    for row in input_rows:
        display(row)
        
    display(set_matrix_btn, output)

In [4]:
def create_matrix_weight() -> None:
    global total_criteria, criteria_labels

    inputs = []
    for i in range(total_alternative):
        inputs.append(widgets.FloatText(value=0, layout=widgets.Layout(width="100px")))
    
    input_rows = []
    for r in range(total_criteria):
        row_widgets = [widgets.Label(value=criteria_labels[r], layout=widgets.Layout(width="100px"))] + [inputs[r]]
        input_rows.append(widgets.HBox(row_widgets))

    def get_matrix():
        return [row.value for row in inputs]

    output = widgets.Output()
    
    def update_matrix(_):
        global weights
        
        weights['weight'] = get_matrix()

        with output:
            print("Weight Criteria:")
            print(weights)

    set_matrix_btn = widgets.Button(description="Set Matrix!")
    set_matrix_btn.on_click(update_matrix)

    for row in input_rows:
        display(row)
        
    display(set_matrix_btn, output)

---

# Input Starter Matrix-Like Data Structure of Criterias and Alternatives

In [5]:
total_criteria = 5
total_alternative = 5

criteria_labels = tuple(f"C{i + 1}" for i in range(total_criteria))
alternative_labels = tuple(f"A{i + 1}" for i in range(total_alternative))

Define criterias

In [6]:
criterias = pd.DataFrame(None, index=criteria_labels, columns=['type'])
print(criterias)

   type
C1  NaN
C2  NaN
C3  NaN
C4  NaN
C5  NaN


Decide which criteria is benefit or cost

In [7]:
create_criteria_decision_form()

Button(description='Set Criteria!', style=ButtonStyle())

Output()

Create matrix for decision

In [8]:
decision_matrix = pd.DataFrame(0, index=alternative_labels, columns=criteria_labels)

print(decision_matrix)

    C1  C2  C3  C4  C5
A1   0   0   0   0   0
A2   0   0   0   0   0
A3   0   0   0   0   0
A4   0   0   0   0   0
A5   0   0   0   0   0


In [9]:
create_matrix_form()

Button(description='Set Matrix!', style=ButtonStyle())

Output()

# Normalization

Formula:

**Benefit criteria:**
$$
r_{ij} = \frac{x_{ij}}{\sqrt{\Sigma_{i=1}^m x^2_{ij}}}
$$

**Cost criteria:**
$$
r_{ij} = 1 - \frac{x_{ij}}{\sqrt{\Sigma_{i=1}^m x^2_{ij}}}
$$

In [10]:
matrix_r = decision_matrix.copy(deep=True).astype(float)

for i in range(total_criteria):
    sum_col = (decision_matrix.iloc[:, i] ** 2).sum()

    for j, row in enumerate(decision_matrix.itertuples(index=False)):
        if criterias.iloc[i]['type'] == 'Benefit':
            matrix_r.iloc[j, i] = decision_matrix.iloc[j, i] / math.sqrt(sum_col) # benefit criteria normalization
        else:
            matrix_r.iloc[j, i] = 1 - (decision_matrix.iloc[j, i] / math.sqrt(sum_col)) # cost criteria normalization

print(matrix_r)

          C1        C2        C3        C4        C5
A1  0.585716  0.395904  0.371232  0.595020  0.395387
A2  0.623378  0.356313  0.472477  0.595020  0.431331
A3  0.548054  0.475085  0.472477  0.561271  0.467275
A4  0.472729  0.475085  0.472477  0.493775  0.431331
A5  0.548054  0.514675  0.438729  0.527523  0.503220


# Define Weights

In [11]:
weights = pd.DataFrame(0, index=criteria_labels, columns=['weight'])
print(weights)

    weight
C1       0
C2       0
C3       0
C4       0
C5       0


In [12]:
create_matrix_weight()

Button(description='Set Matrix!', style=ButtonStyle())

Output()

# Calculate Weighted Matrix

Formula:

$$
v_{ij} = r_{ij} \cdot w_{ij}
$$

In [13]:
weighted_matrix = matrix_r.mul(weights['weight'], axis=1)
print(weighted_matrix)

          C1        C2        C3        C4        C5
A1  3.514296  5.542653  2.227392  7.735255  4.744642
A2  3.740269  4.988388  2.834863  7.735255  5.175973
A3  3.288323  6.651184  2.834863  7.296526  5.607304
A4  2.836377  6.651184  2.834863  6.419069  5.175973
A5  3.288323  7.205449  2.632372  6.857798  6.038635


# Get The Concordance and Discordance Set

Calculate the permutation of alternative to see how many group should concordance and discordance have

In [14]:
permutation_alternative = math.perm(total_alternative, 2)
print(permutation_alternative)

20


## Concordance

In [15]:
concordances_data = []
for a, b in permutations(alternative_labels, 2):
    row_1 = int(a[-1]) - 1
    row_2 = int(b[-1]) - 1

    selected_criteria = []
    for i in range(len(weighted_matrix.columns)):
        if weighted_matrix.iloc[row_1, i] >= weighted_matrix.iloc[row_2, i]:
            selected_criteria.append([a, b, f"C{i+1}"])

    is_a_found = any(row[0] == a for row in selected_criteria)
    is_b_found = any(row[1] == b for row in selected_criteria)

    if is_a_found and is_b_found:
        for i in selected_criteria:
            concordances_data.append(i)
    else:
        concordances_data.append([a, b, None])

concordances = pd.DataFrame(concordances_data, columns=['First', 'Second', 'Selected Criteria'])
print(concordances)

   First Second Selected Criteria
0     A1     A2                C2
1     A1     A2                C4
2     A1     A3                C1
3     A1     A3                C4
4     A1     A4                C1
5     A1     A4                C4
6     A1     A5                C1
7     A1     A5                C4
8     A2     A1                C1
9     A2     A1                C3
10    A2     A1                C4
11    A2     A1                C5
12    A2     A3                C1
13    A2     A3                C3
14    A2     A3                C4
15    A2     A4                C1
16    A2     A4                C3
17    A2     A4                C4
18    A2     A4                C5
19    A2     A5                C1
20    A2     A5                C3
21    A2     A5                C4
22    A3     A1                C2
23    A3     A1                C3
24    A3     A1                C5
25    A3     A2                C2
26    A3     A2                C3
27    A3     A2                C5
28    A3     A

## Discordance

In [16]:
discordances_data = []
for a, b in permutations(alternative_labels, 2):
    row_1 = int(a[-1]) - 1
    row_2 = int(b[-1]) - 1

    selected_criteria = []
    for i in range(len(weighted_matrix.columns)):
        if weighted_matrix.iloc[row_1, i] < weighted_matrix.iloc[row_2, i]:
            selected_criteria.append([a, b, f"C{i+1}"])

    is_a_found = any(row[0] == a for row in selected_criteria)
    is_b_found = any(row[1] == b for row in selected_criteria)

    if is_a_found and is_b_found:
        for i in selected_criteria:
            discordances_data.append(i)
    else:
        discordances_data.append([a, b, None])

discordances = pd.DataFrame(discordances_data, columns=['First', 'Second', 'Selected Criteria'])
print(discordances)

   First Second Selected Criteria
0     A1     A2                C1
1     A1     A2                C3
2     A1     A2                C5
3     A1     A3                C2
4     A1     A3                C3
5     A1     A3                C5
6     A1     A4                C2
7     A1     A4                C3
8     A1     A4                C5
9     A1     A5                C2
10    A1     A5                C3
11    A1     A5                C5
12    A2     A1                C2
13    A2     A3                C2
14    A2     A3                C5
15    A2     A4                C2
16    A2     A5                C2
17    A2     A5                C5
18    A3     A1                C1
19    A3     A1                C4
20    A3     A2                C1
21    A3     A2                C4
22    A3     A4              None
23    A3     A5                C2
24    A3     A5                C5
25    A4     A1                C1
26    A4     A1                C4
27    A4     A2                C1
28    A4     A

# Define The Matrix of Concordance and Discordance

## Concordance

Formula

$$
c_{kl} = \Sigma_{j \in c_{kl}} W_j
$$

In [17]:
sum_weights_concordance = {f"{a[-1]}-{b[-1]}" : 0 if a != b else None for a, b in product(alternative_labels, repeat=2)}

for val in concordances.itertuples(index=False):
    sum_weights_concordance[val[0][-1]+'-'+val[1][-1]] += weights.loc[val[2]].weight

print(json.dumps(sum_weights_concordance, indent=4))

{
    "1-1": null,
    "1-2": 27.0,
    "1-3": 19.0,
    "1-4": 19.0,
    "1-5": 19.0,
    "2-1": 37.0,
    "2-2": null,
    "2-3": 25.0,
    "2-4": 37.0,
    "2-5": 25.0,
    "3-1": 32.0,
    "3-2": 32.0,
    "3-3": null,
    "3-4": 51.0,
    "3-5": 25.0,
    "4-1": 32.0,
    "4-2": 32.0,
    "4-3": 20.0,
    "4-4": null,
    "4-5": 6.0,
    "5-1": 32.0,
    "5-2": 26.0,
    "5-3": 32.0,
    "5-4": 45.0,
    "5-5": null
}


Convert it to matrix-like data structure (Pandas DataFrame)

In [18]:
last_dict_item_concordance = list(sum_weights_concordance.keys())[-1]

concordance_matrix = pd.DataFrame(np.nan, index=range(int(last_dict_item_concordance[0])), columns=range(int(last_dict_item_concordance[-1])))

for key, value in sum_weights_concordance.items():
    row, col = map(int, key.split('-'))
    concordance_matrix.at[row-1, col-1] = value
    
print(concordance_matrix)

      0     1     2     3     4
0   NaN  27.0  19.0  19.0  19.0
1  37.0   NaN  25.0  37.0  25.0
2  32.0  32.0   NaN  51.0  25.0
3  32.0  32.0  20.0   NaN   6.0
4  32.0  26.0  32.0  45.0   NaN


/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in greater
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()


## Discordance

Formula

$$
d_{kl} = \frac{max(|v_{kj} - v_{vj}|)_{j \in D_{kl}}}{max(|v_{kj} - v_{lj}|)_{\forall_{j}}}
$$

In [20]:
discordances_group = {f"{a[-1]}-{b[-1]}" : [] if a != b else None for a, b in product(alternative_labels, repeat=2)}
print(json.dumps(discordances_group, indent=4))

{
    "1-1": null,
    "1-2": [],
    "1-3": [],
    "1-4": [],
    "1-5": [],
    "2-1": [],
    "2-2": null,
    "2-3": [],
    "2-4": [],
    "2-5": [],
    "3-1": [],
    "3-2": [],
    "3-3": null,
    "3-4": [],
    "3-5": [],
    "4-1": [],
    "4-2": [],
    "4-3": [],
    "4-4": null,
    "4-5": [],
    "5-1": [],
    "5-2": [],
    "5-3": [],
    "5-4": [],
    "5-5": null
}


In [21]:
for val in discordances.itertuples(index=False):
    a = val[0][-1]
    b = val[1][-1]
    discordances_group[a+'-'+b].append(val[2])

print(json.dumps(discordances_group, indent=4))

{
    "1-1": null,
    "1-2": [
        "C1",
        "C3",
        "C5"
    ],
    "1-3": [
        "C2",
        "C3",
        "C5"
    ],
    "1-4": [
        "C2",
        "C3",
        "C5"
    ],
    "1-5": [
        "C2",
        "C3",
        "C5"
    ],
    "2-1": [
        "C2"
    ],
    "2-2": null,
    "2-3": [
        "C2",
        "C5"
    ],
    "2-4": [
        "C2"
    ],
    "2-5": [
        "C2",
        "C5"
    ],
    "3-1": [
        "C1",
        "C4"
    ],
    "3-2": [
        "C1",
        "C4"
    ],
    "3-3": null,
    "3-4": [
        null
    ],
    "3-5": [
        "C2",
        "C5"
    ],
    "4-1": [
        "C1",
        "C4"
    ],
    "4-2": [
        "C1",
        "C4"
    ],
    "4-3": [
        "C1",
        "C4",
        "C5"
    ],
    "4-4": null,
    "4-5": [
        "C1",
        "C2",
        "C4",
        "C5"
    ],
    "5-1": [
        "C1",
        "C4"
    ],
    "5-2": [
        "C1",
        "C3",
        "C4"
    ],
    "5-3": [
 

In [39]:
discordance_matrix_data = {f"{a[-1]}-{b[-1]}" : 0 if a != b else None for a, b in product(alternative_labels, repeat=2)}

for key, val in discordances_group.items():
    if val == None: continue
        
    a1 = f"A{key[0]}"
    a2 = f"A{key[-1]}"

    difference_criteria = []
    difference_criteria_discordance = []
    for criteria in weighted_matrix.columns:
        if criteria in val:
            difference_criteria_discordance.append(abs(weighted_matrix.loc[a1][criteria] - weighted_matrix.loc[a2][criteria]))
        else:
            difference_criteria.append(abs(weighted_matrix.loc[a1][criteria] - weighted_matrix.loc[a2][criteria]))

    discordance_matrix_data[key] = max(difference_criteria_discordance, default=float('-inf')) / max(difference_criteria, default=float('-inf'))

print(json.dumps(discordance_matrix_data, indent=4))

{
    "1-1": null,
    "1-2": 1.0959924207586793,
    "1-3": 2.5266878828539916,
    "1-4": 0.8422292942846661,
    "1-5": 1.8950159121404964,
    "2-1": 0.9124150688083862,
    "2-2": null,
    "2-3": 3.6791904009264913,
    "2-4": 1.2633439414269985,
    "2-5": 2.5266878828539947,
    "3-1": 0.3957750408295232,
    "3-2": 0.27179892613010204,
    "3-3": null,
    "3-4": -Infinity,
    "3-5": 1.263343941426999,
    "4-1": 1.1873251224885664,
    "4-2": 0.7915500816590447,
    "4-3": Infinity,
    "4-4": null,
    "4-5": 4.260266517605857,
    "5-1": 0.5277000544393635,
    "5-2": 0.3957750408295227,
    "5-3": 0.7915500816590445,
    "5-4": 0.2347270988487289,
    "5-5": null
}


<ipython-input-39-372d74f1ca16>:17: RuntimeWarning: divide by zero encountered in scalar divide
  discordance_matrix_data[key] = max(difference_criteria_discordance, default=float('-inf')) / max(difference_criteria, default=float('-inf'))


Convert data structure to matrix-like data type (Pandas DataFrame)

In [40]:
last_dict_item_discordance = list(discordance_matrix_data.keys())[-1]

discordance_matrix = pd.DataFrame(np.nan, index=range(int(last_dict_item_discordance[0])), columns=range(int(last_dict_item_discordance[-1])))

for key, value in discordance_matrix_data.items():
    row, col = map(int, key.split('-'))
    discordance_matrix.at[row-1, col-1] = value
    
print(discordance_matrix)

          0         1         2         3         4
0       NaN  1.095992  2.526688  0.842229  1.895016
1  0.912415       NaN  3.679190  1.263344  2.526688
2  0.395775  0.271799       NaN      -inf  1.263344
3  1.187325  0.791550       inf       NaN  4.260267
4  0.527700  0.395775  0.791550  0.234727       NaN


/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in greater
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()


# Determine the Dominant Aggregate of The Matrix

In [ ]:
print('Calculation should be here')

# Eliminate Less Favourable Alternatives

In [ ]:
print('Calculation should be here')

# Rangking Alternative

In [ ]:
print('Calculation should be here')

---

# Conclusion